Summary 
To what extent has there been a change in confidence between political leaders and types of news due to SARS-CoV-2. The subset of GDELT data that will be used to answer this question is the Event table with Actor1Name, Actor1CountryCode and EventCode as the columns. Market basket analysis including A-priori algorithm will be used and the intended result will return the actor and event tuple with confidence for each record this will include all countries and those specifically defined in the rules below. The data will be within the date ranges 31/08/19 - 30/12/19 for pre SARS-CoV-2 and 31/12/19 - 31/04/20 for post SARS-CoV-2. This result will allow me to make a simple comparison between the two date ranges and in turn answer the question.


Event table with Actor1Name, Actor1CountryCode and EventCode as the columns31/08/19 - 30/12/19 for pre SARS-CoV-2 and 31/12/19 - 31/04/20 for post SARS-CoV-2. 

#Imports and installs required for project

In [1]:
#library and code setup - retrived from Sample GDELT starter only.ipynb "Written by James Atlas"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark
!pip install newspaper3k
!pip install gdelt

#library and code setup
import pyspark, os
from pyspark import SparkConf, SparkContext
os.environ["PYSPARK_PYTHON"]="python3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

     |████████████████████████████████| 217.8MB 60kB/s 
     |████████████████████████████████| 204kB 47.6MB/s 
     |████████████████████████████████| 215kB 4.7MB/s 
     |████████████████████████████████| 7.4MB 8.1MB/s 
     |████████████████████████████████| 194kB 46.3MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-cp36-none-any.whl size=13539 sha256=fcd9c8f64ddfe75a334e837aec09ecc1099cdd62f10f78506289ab320e6230e0
  Stored in directory: /root/.cache/pip/wheels/81/2b/43/a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Created wheel for jieba3k: filename=jieba3k-0.35.1-cp36-none-any.whl size=7398406 sha256=eaf27207e4ad45d488fec6f3dd08ea6fb38a4c43c8283e47ab150ca89088dc86
  Stored in directory: /root/.cache/pip/wheels/83/15/9c/a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=ce897104b976cb3a7358d4d95033ca005e7f70b25

In [0]:
#start spark local server - retrived from Sample GDELT starter only.ipynb "Written by James Atlas"
import sys, os
from operator import add
import time

os.environ["PYSPARK_PYTHON"]="python3"

import pyspark
from pyspark import SparkConf, SparkContext

#connects our python driver to a local Spark JVM running on the Google Colab server virtual machine
try:
  conf = SparkConf().setMaster("local[*]").set("spark.executor.memory", "1g")
  sc = SparkContext(conf = conf)
except ValueError:
  #it's ok if the server is already started
  pass

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
    print(x)

In [3]:
# these remove files from previous runs - retrived from Sample GDELT starter only.ipynb "Written by James Atlas"
!rm -rf articles
!rm *.csv

rm: cannot remove '*.csv': No such file or directory


Function to generate all the time slots required to check hourly via api

In [0]:
def create_time():
  """basic loop to get all the times we need to call later """
  time_loop = []
  hour_start = 000000
  for i in range(0, 24):
          """simple loops to calculate all base time for hours for api call per hour """
          if i == 0:
                  time_loop.append(("000000", "005959"))
              
          elif i <= 9:
              hour_start = 0
              hour_start =  hour_start + i * 10000
              time_loop.append(("0" + str(hour_start), "0" + str(hour_start+5959)))

          else:
              hour_start = 0
              hour_start =  hour_start + i * 10000
              time_loop.append((str(hour_start),str(hour_start+5959)))
  return time_loop

In [0]:
import timeit
import requests
import csv 
import pandas as pd

def Api_Hourly_call(date_to_download, time_loop, queryURLbase, filename):
  """Makes an api call 24 times for each hour of the day then merges them into a single csv file.
  checks for duplicates of articles/reposts and condenses in a single file takes about 30 mins all up for 4 months of records """

  from more_itertools import unique_everseen
  #run through the day hour by hour to get all the csv files downloaded and combined 
  for d_time in time_loop:
    startdate = date_to_download + d_time[0] #0 hour 0 min 0 sec (start of day)
    enddate = date_to_download + d_time[1]#23 hour 59 min 59 sec (end of day)
    query = queryURLbase + "&startdatetime=" + startdate + "&enddatetime=" + enddate

    #downloads the new csv file for the specificed hour
    req = requests.get(query)
    url_content = req.content
    csv_file = open('downloaded.csv', 'wb')
    csv_file.write(url_content)
    csv_file.close()

    try:
      #combines the csv files and checks for duplicates 
      df1 = pd.read_csv(filename)
      df2 = pd.read_csv('downloaded.csv')
      df_row = pd.concat([df1, df2])
      df_row.drop_duplicates(subset=None, inplace=True) #sometimes get duplicates on the hour this removes them
      df_row.to_csv(filename, index=False)


      # print("start time: " + d_time[0] + " to end time: " + d_time[1] + " has been added to output file.")
      # print(query)
      # print("added:")
    except:
      # print("Empty File no records at this time omitted from combination: ", query)
      pass



#code to time a function
# start = timeit.default_timer()
# stop = timeit.default_timer()
# print('Time taken to download all files: ', stop - start) 



#Generates the list of dates needed for API in correct format

In [0]:
def long_enough(date):
  """helper function for getting correct date format """
  if date <= 9:
    date = "0" + str(date)
  return date

def get_date_range(date_start, date_end):
  """Take two dates input start date and end date and generates all the dates in between in usable format for API """
  d_lst = []
  date_range = [d_lst.append(str(x.year)+ str(long_enough(x.month)) + str(long_enough(x.day))) for x in pd.date_range(date_start, date_end)]
  return d_lst

#NZ 31/01/19 to 30/04/2020 keywords inc corona virus and covid 19 etc..

In [0]:
#gets the data for during the corona virus period average run time 30 - 35 mins for 4 months of records hourly 
import pandas as pd
import csv
import timeit
start = timeit.default_timer()

#API I want to download CSV file from includes my key words
URL_OF_REQUEST = "https://api.gdeltproject.org/api/v2/doc/doc?format=html&query=(%22Coronavirus%22%20OR%20%22Covid-19%22%20OR%20%22Sars-Cov-19%22%20OR%20%22chinese%20virus%22%20OR%20%22chinese%20flu%22%20OR%20%22SARS-CoV-2%22)%20sourcecountry:NZ&mode=artlist&maxrecords=250&format=csv&sort=hybridrel"
#Create all the times in 24 hours
time_loop = create_time()
#start a counter to see remaining days left to process
count = 0
#get the date range for the specific dates we want in API format
date_range = get_date_range('2019-12-31', '2020-04-30') # 2019-12-31
#set the file name we want our condensed file as 
filename = "Coronavirus_records_unfiltered.csv"
#just a simple variable to see how many days in total we are doing to see how many left
total = len(date_range)
#generate an empty csv file as to not error at start
df = pd.DataFrame(list())
df.to_csv(filename)

#loop through all dates calling the API on every date combining each time
for date in date_range:
  Api_Hourly_call(date, time_loop, URL_OF_REQUEST, filename)
  print("completed date: ", date)
  count = count + 1
  print(str(count) + " of " + str(total) + " remaining")
stop = timeit.default_timer()
print('Time taken to download all files: ', stop - start) 

completed date:  20191231
1 of 122 remaining
completed date:  20200101
2 of 122 remaining
completed date:  20200102
3 of 122 remaining
completed date:  20200103
4 of 122 remaining
completed date:  20200104
5 of 122 remaining
completed date:  20200105
6 of 122 remaining
completed date:  20200106
7 of 122 remaining
completed date:  20200107
8 of 122 remaining
completed date:  20200108
9 of 122 remaining
completed date:  20200109
10 of 122 remaining
completed date:  20200110
11 of 122 remaining
completed date:  20200111
12 of 122 remaining
completed date:  20200112
13 of 122 remaining
completed date:  20200113
14 of 122 remaining
completed date:  20200114
15 of 122 remaining
completed date:  20200115
16 of 122 remaining
completed date:  20200116
17 of 122 remaining
completed date:  20200117
18 of 122 remaining
completed date:  20200118
19 of 122 remaining
completed date:  20200119
20 of 122 remaining
completed date:  20200120
21 of 122 remaining
completed date:  20200121
22 of 122 remaini

In [0]:
#API to get the articles with the SPECIFIC NZ actors I need as Actor1name does not give the auctual name as I assume...
#gets the data for during the corona virus period average run time 30 - 35 mins for 4 months of records hourly 
import pandas as pd
import csv 
import timeit
start = timeit.default_timer()
#API I want to download CSV file from includes my key words
#Looks like I need to get each actor individually so do 3 invidual api calls RIP :(
NZ_Corona_URL = "https://api.gdeltproject.org/api/v2/doc/doc?format=html&query=(Jacinda%20Ardern)sourcecountry:NZ&mode=artlist&maxrecords=250&format=csv&sort=hybridrel"
#"https://api.gdeltproject.org/api/v2/doc/doc?format=html&query=(%22David%20Clark%22%20OR%20%22Jacinda%20Ardern%22%20OR%20%22Ashley%20Bloomfield%22)sourcecountry:NZ&mode=artlist&maxrecords=250&format=csv&sort=hybridrel"
#Create all the times in 24 hours
time_loop = create_time()
#start a counter to see remaining days left to process
count = 0
#get the date range for the specific dates we want in API format
date_range = get_date_range('2019-12-31', '2020-04-30') # 2019-12-31
#set the file name we want our condensed file as 
filename = "Jacinda_Ardern_All_files.csv"
#just a simple variable to see how many days in total we are doing to see how many left
total = len(date_range)
#generate an empty csv file as to not error at start
df = pd.DataFrame(list())
df.to_csv(filename)

#loop through all dates calling the API on every date combining each time
for date in date_range:
  Api_Hourly_call(date, time_loop, NZ_Corona_URL, filename)
  print("completed date: ", date)
  count = count + 1
  print(str(count) + " of " + str(total) + " remaining")
stop = timeit.default_timer()
print('Time taken to download all files: ', stop - start) 

#All data from range, dont need to use the API for this.

moving on to fetching gdelt data from table 

In [0]:
#James Atlas code with dates changed - just downloads all the gdelt data I need from the tables
from concurrent.futures import ProcessPoolExecutor
from datetime import date, timedelta
import pandas as pd
import gdelt
import os

# set up gdeltpyr for version 2
gd = gdelt.gdelt(version=2)

# multiprocess the query
e = ProcessPoolExecutor()


# generic functions to pull and write data to disk based on date
def get_filename(x):
  date = x.strftime('%Y%m%d')
  return "{}_gdeltdata.csv".format(date)

def intofile(filename):
    try:
        if not os.path.exists(filename):
          date = filename.split("_")[0]
          d = gd.Search(date, table='events',coverage=False) #not updata at 15mins
          d.to_csv(filename,encoding='utf-8',index=False)
    except:
        print("Error occurred")

# pull the data from gdelt into multi files; this may take a long time
dates = [get_filename(x) for x in pd.date_range('2019-12-31','2020-04-30')]

results = list(e.map(intofile,dates))


Combine all the Gdeltdata into a single CSV file

In [0]:
from pyspark.sql import SQLContext
file_format = "*_gdeltdata"
sqlContext = SQLContext(sc)
data_urls = sqlContext.read.option("header", "true").csv(dates)
data_urls = data_urls[['Actor1Name','Actor2Name','Actor1CountryCode', 'EventCode']]
data_urls.write.csv('before_data_unfiltered.csv') #this merges it in to a folder of 5 csv which acts as a single csv file

Do a join to merge NZ_Actors_By_Name_during_coronavirus and coronavirus_records_unfiltered This will give a table that contains all the records that are about one of these individuals and related to corona virus.


In [0]:
data_frame_virus = pd.read_csv('Coronavirus_records_unfiltered.csv')
data_frame_actors = pd.read_csv('NZ_Actors_By_Name_during_coronavirus.csv')
display(data_frame_virus)
display(data_frame_actors)
